## Notion API 연결

In [1]:
import os
import boto3
from dotenv import load_dotenv
from notion_client import Client
import urllib.parse

load_dotenv('../.env')
NOTION_DATABASE_ID = os.getenv("NOTION-DB")
NOTION_API_KEY = os.getenv("NOTION")
AWS_ACCESS_KEY = os.getenv("AWS-ACCESS-KEY")
AWS_SECRET_KEY = os.getenv("AWS-SECRET-KEY")
S3_BUCKET_NAME = os.getenv("S3-BUCKET-NAME")
S3_REGION = os.getenv("S3-BUCKET-REGION")

### 파일 업로드 함수

In [2]:
def upload_to_s3(file_path, file_name):
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=S3_REGION,
    )
    try:
        encoded_file_name = urllib.parse.quote(file_name)
        # S3에 파일 업로드
        s3_client.upload_file(
            file_path, 
            S3_BUCKET_NAME, 
            file_name,
            # ExtraArgs={"ACL": "public-read"}
        )
        # S3 URL 생성
        file_url = f"https://{S3_BUCKET_NAME}.s3.{S3_REGION}.amazonaws.com/{encoded_file_name}"
        return file_url
    except Exception as e:
        print("S3 업로드 오류:", e)
        return None

### DB 적재 함수

In [3]:
def add_receipt_to_notion(data, file_url):
    notion = Client(auth=NOTION_API_KEY)
    try:
        file_name = file_url.split("/")[-1]  # URL의 마지막 부분을 파일 이름으로 사용
        encoded_file_name = urllib.parse.quote(file_name)
        response = notion.pages.create(
            parent={"database_id": NOTION_DATABASE_ID},
            properties={
                "상호명": {"title": [{"text": {"content": data["상호명"]}}]},
                "날짜": {"date": {"start": data["날짜"]}},
                "총액": {"number": data["총액"]},
                "영수증": {
                    "files": [
                        {
                            "type": "external", 
                            "external": {"url": file_url},
                            "name": encoded_file_name  # 파일 이름 설정
                        }
                    ]
                }
            },
            children=[
                {
                    "object": "block",
                    "type": "bulleted_list_item",
                    "bulleted_list_item": {
                        "rich_text": [
                            {
                                "type": "text", 
                                "text": {"content": f"{item['이름']}: {item['가격']}원"}
                            }
                        ]
                    },
                }
                for item in data["항목"]
            ]
        )
        print("데이터가 성공적으로 Notion에 저장되었습니다:", response["id"])
    except Exception as e:
        print("Notion 저장 오류:", e)

In [4]:
notion = Client(auth=NOTION_API_KEY)

# 영수증 파일 경로
receipt_file_path = "../image/air-korean.pdf"
receipt_file_name = "air-korean.pdf"  # S3에 저장될 파일 이름
# 1. S3에 파일 업로드 및 URL 생성
s3_file_url = upload_to_s3(receipt_file_path, receipt_file_name)
print("S3 파일 URL:", s3_file_url)
if s3_file_url:
    # 2. ChatGPT로 추출된 영수증 데이터 (예제)
    extracted_data = {
        "상호명": "대한항공",
        "날짜": "2023-12-27",
        "항목": [
            {"이름": "상품1", "가격": 10000},
            {"이름": "상품2", "가격": 5000},
        ],
        "총액": 15000,
    }
    # 3. Notion 데이터베이스에 데이터 저장
    add_receipt_to_notion(extracted_data, s3_file_url)

S3 파일 URL: https://receipt-bucket-1227.s3.ap-southeast-2.amazonaws.com/air-korean.pdf
데이터가 성공적으로 Notion에 저장되었습니다: 17d81589-428d-81db-b131-d67693140476
